# VOC to YOLO

The BCCD dataset co-ordinates are in PASCAL VOC format.  
PASCAL VOC bounding box cordinates are :  
(xmin, ymin, xmax, ymax) i.e co-ordinate of top left (xmin, ymin) and bottom right(xmax, ymax) position of the box.  
YOLOv3 bounding box co-ordinate format is:
(x, y, w, h) , i.e. center of the bounding box (x,y) and width (w) , height (h)
In addition the co-ordinates as well as height and width are required to be normalized according to oriiginal image height and width

In [10]:
# Connect with google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Accesss training data bounding boxes

In [17]:
import util
import numpy as np
import os

In [44]:
# Load adta
BCCDdf, _ = util.load_data('/content/gdrive/MyDrive/UpGrad/LJMU MS /Data/YOLO_00/YOLO/Training/Annotations' )

Convert to YOLO annotation format

In [61]:
# Encode cell types to numeric 0 - RBC , 1- WBC and 2- Platelet
BCCDdf['cell_label'] = np.where(BCCDdf['cell_type']=='RBC', 0, 
                        np.where(BCCDdf['cell_type']=='WBC', 1,2))

# Calculate co-ordinates according to YOLO v3 training and round upto 6 digits
# x: bounding box center x co-ordinate
BCCDdf['x'] = (BCCDdf['xmin'] + BCCDdf['xmax']) / (640*2)
BCCDdf['x'] = BCCDdf['x'].round(6)
#bounding box center y co-ordinate
BCCDdf['y'] = (BCCDdf['ymin'] + BCCDdf['ymax']) / (480*2)
BCCDdf['y'] = BCCDdf['y'].round(6)
# bounding box width
BCCDdf['w'] = (BCCDdf['xmax'] - BCCDdf['xmin']) / 640
BCCDdf['w'] = BCCDdf['w'].round(6)
#bounding box height
BCCDdf['h'] = (BCCDdf['ymax'] - BCCDdf['ymin']) / 480
BCCDdf['h'] = BCCDdf['h'].round(6)



Create a single annotation file for each image and store it in the same location as image

In [67]:
train_IDs=list(set(list(BCCDdf['ID'])))

In [88]:
gdrive_path= '/content/gdrive/MyDrive/UpGrad/LJMU MS /Data/YOLO_00/YOLO/Training/Images'

for id in train_IDs:
  # Collect annotations for an image
  tempdf = BCCDdf[BCCDdf['ID']==id]
  tempdf = tempdf[['cell_label','x','y','w','h']]
  #Path to store te annotation
  file_ext = list(BCCDdf[BCCDdf['ID']==id]['filename'])[0].split('/')[-1].split('.')[0]
  file_path= os.path.join(gdrive_path,file_ext)
  #store annotation in gdrive
  tempdf.to_csv(os.path.join(file_path+".txt"), index = False, header=False, sep=' ')
  